# CS5228-2410 Final Project EDA

## Overview

*(from [Kaggle Project](https://www.kaggle.com/competitions/cs-5228-2410-final-project))*

In this project, we look into the market for used cars in Singapore. Car ownership in Singapore is rather expensive which includes the very high prices for new and used cars (compared to many other countries). There are many stakeholders in this market. Buyers and sellers want to find good prices, so they need to understand what affects the value of a car. Online platforms facilitating the sale of used cars, on the other hand, want to maximize the number of sales/transactions.

The goal of this task is to predict the resale price of a car based on its properties (e.g., make, model, mileage, age, power, etc). It is therefore first and foremost a regression task. These different types of information allow you to come up with features for training a regressor. It is part of the project for you to justify, derive and evaluate different features. Besides predicting the outcome in terms of a dollar value, other useful results include the importance of different attributes, the evaluation and comparison of different regression techniques, an error analysis and discussion about limitations and potential extensions, etc.

## Main Steps
1. Load dataset
2. Initial inspection
    - Basic information
    - Summary stats
3. Data quality check
    - Missing values
    - Identify duplicates
    - Examine data types
4. EDA
    - Univariate analysis
        - Numerical: histograms, boxplots
        - Categorical: barplots, pie charts
    - Bivariate analysis
        - Scatterplots: price v numerical
        - Boxplots: price v categorical
    - Correlation analysis

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import difflib
import dateutil
from datetime import datetime
from wordcloud import WordCloud
# from sklearn.cluster import DBSCAN
# from sklearn.preprocessing import StandardScaler
# import category_encoders as ce
# from sklearn.impute import KNNImputer

# from carly.dataproc import *

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
if not os.path.exists('visualisations/'):
    os.mkdir('visualisations')

## 1    Load data

In [4]:
df_train = pd.read_csv("dataset/train.csv")
df_test = pd.read_csv("dataset/test.csv")

## 2    Initial Inspection

In [5]:
print("# of records: ",len(df_train))
print("# of columns: ",len(df_train.columns))

# of records:  25000
# of columns:  30


In [6]:
print("# of records: ",len(df_test))
print("# of columns: ",len(df_test.columns))

# of records:  10000
# of columns:  29


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   listing_id         25000 non-null  int64  
 1   title              25000 non-null  object 
 2   make               23684 non-null  object 
 3   model              25000 non-null  object 
 4   description        24320 non-null  object 
 5   manufactured       24993 non-null  float64
 6   original_reg_date  255 non-null    object 
 7   reg_date           25000 non-null  object 
 8   type_of_vehicle    25000 non-null  object 
 9   category           25000 non-null  object 
 10  transmission       25000 non-null  object 
 11  curb_weight        24693 non-null  float64
 12  power              22360 non-null  float64
 13  fuel_type          5879 non-null   object 
 14  engine_cap         24404 non-null  float64
 15  no_of_owners       24982 non-null  float64
 16  depreciation       244

In [8]:
print(df_train.head(3))

   listing_id                                              title  \
0     1292132  Land Rover Range Rover Velar 3.0A Si6 R-Dynami...   
1     1294696   Mercedes-Benz C-Class C200 Sport Premium Sunroof   
2     1311717              Honda Odyssey 2.4A (COE till 09/2027)   

            make    model                                        description  \
0     land rover    range  1 owner, no repairs needed! it looks great, in...   
1  mercedes-benz     c200  rare beautiful white c200 sport premium sunroo...   
2          honda  odyssey            comes with warranty. full service done.   

   manufactured original_reg_date     reg_date type_of_vehicle  \
0        2018.0               NaN  08-mar-2018             suv   
1        2017.0               NaN  28-dec-2017    luxury sedan   
2        2007.0               NaN  19-sep-2007             mpv   

                                   category transmission  curb_weight  power  \
0                                  parf car         auto     

## 3    Data Quality Check

In [9]:
# Calculate missing values
def get_missing_info(df_train):
    missing_values = df_train.isnull().sum()
    missing_percentages = (missing_values / len(df_train) * 100).round(2)

    missing_info = pd.DataFrame({
        'missing_count': missing_values,
        'missing_perc': missing_percentages
    }).sort_values(by="missing_count", ascending=False)

    return missing_info

missing_info = get_missing_info(df_train=df_train)
print("\n=== missing_count Analysis ===")
print(missing_info[missing_info['missing_count'] > 0].sort_values('missing_perc', ascending=False))



=== missing_count Analysis ===
                   missing_count  missing_perc
indicative_price           25000        100.00
opc_scheme                 24838         99.35
original_reg_date          24745         98.98
lifespan                   22671         90.68
fuel_type                  19121         76.48
mileage                     5304         21.22
accessories                 3813         15.25
power                       2640         10.56
road_tax                    2632         10.53
make                        1316          5.26
features                     843          3.37
description                  680          2.72
engine_cap                   596          2.38
depreciation                 507          2.03
curb_weight                  307          1.23
dereg_value                  220          0.88
arf                          174          0.70
omv                           64          0.26
no_of_owners                  18          0.07
manufactured                

The dataset has significant missing data across several features:

- 'indicative_price' is completely missing (100%)
- 'opc_scheme' and 'original_reg_date' are missing for nearly all records (>98%)
- 'lifespan' is missing for about 90% of the records
- 'fuel_type' is missing for about 76% of the records
- 'mileage' is missing for about 21% of the records
- Several other fields have missing data ranging from 0.03% to 15.25%


In [10]:
print("\n=== Duplicate Records Analysis ===")
print(f"Number of duplicate rows: {df_train.duplicated().sum()}")
print(f"Number of duplicate listing_ids: {df_train['listing_id'].duplicated().sum()}")


=== Duplicate Records Analysis ===
Number of duplicate rows: 0
Number of duplicate listing_ids: 0



There are no duplicate rows or listing_ids in the dataset.

In [11]:
df_train.sample(3)

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
9262,1307886,Honda Vezel 1.5A G,NaN,vezel,NaN,2021.0,NaN,16-aug-2021,suv,"parf car, premium ad car, sgcarmart warranty cars",auto,1250.0,87.0,NaN,1496.0,2.0,16390.0,45189,682.0,40401.0,59000.0,24048.0,10668.0,NaN,NaN,uncategorized,view specs of the honda vezel,NaN,NaN,122900.0
478,1244805,BMW 5 Series 530e Plug-in Hybrid,bmw,530e,luxury executive sedan in white. genuine low m...,2017.0,NaN,12-feb-2018,luxury sedan,"parf car, direct owner sale, low mileage car, ...",auto,1770.0,185.0,petrol-electric,1998.0,2.0,26480.0,42322,1210.0,54523.0,45000.0,54462.0,60032.0,NaN,NaN,uncategorized,"2.0l bmw twinpower turbo, 248bhp and 350nm of ...",25000,NaN,127000.0
8238,1294909,Honda Vezel 1.5A X Honda Sensing,honda,vezel,0,2016.0,NaN,04-oct-2016,suv,"parf car, sgcarmart warranty cars",auto,1190.0,96.0,NaN,1496.0,2.0,15460.0,52301,682.0,19575.0,NaN,21831.0,12564.0,NaN,NaN,uncategorized,NaN,NaN,NaN,41900.0


**Observations**
- Since `indicative_price` is completely missing, drop column. 
- `listing_id` is not meaningful to the analysis as well. 
- Since `original_reg_date` is almost entirely missing, feature is not meaningful. Based on context, last `reg_date` may be more useful as it may be closely related to the COE price upon time of registration. COE has a heavy influence on car resale price.
- `fuel_type` has many missing values but can potentially be obtained from `category`.
- `lifespan` has many missing values but can potentially be inferred from the `title`.

In [12]:
COLS_TO_DROP = ['indicative_price']

df_train = df_train.drop(columns=COLS_TO_DROP)
df_train.columns

Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'price'],
      dtype='object')

#### Basic Sanity Check

In [13]:
# Check model make quality
unique_makes = df_train['make'].unique()
unique_models = df_train['model'].unique()

make_model_pairs = df_train.groupby('make')['model'].apply(set)
model_count = df_train['model'].value_counts()
make_count = df_train['make'].value_counts()

# 4. Check for missing values
missing_makes = df_train['make'].isnull().sum()
missing_models = df_train['model'].isnull().sum()

# 5. Text normalization
df_train['make'] = df_train['make'].str.strip().str.title()  # Example normalization
df_train['model'] = df_train['model'].str.strip().str.title()

# Output the results
print(f"Unique Makes: {unique_makes}")
print(f"Unique Models: {unique_models}")
print("Make-Model Pairs:")
print(make_model_pairs)
print(f"Missing Makes: {missing_makes}, Missing Models: {missing_models}")
print(model_count[:10])
print(make_count[:10])

Unique Makes: ['land rover' 'mercedes-benz' 'honda' 'toyota' 'lexus' 'lamborghini'
 'hyundai' 'mitsubishi' 'subaru' 'bmw' 'audi' 'mazda' nan 'peugeot'
 'volkswagen' 'bentley' 'ssangyong' 'isuzu' 'volvo' 'nissan' 'kia'
 'jaguar' 'mg' 'seat' 'mini' 'citroen' 'ferrari' 'porsche' 'tesla'
 'rolls-royce' 'suzuki' 'infiniti' 'chevrolet' 'opel' 'renault' 'mclaren'
 'morris' 'byd' 'fiat' 'iveco' 'skoda' 'maserati' 'dfsk' 'perodua' 'hino'
 'daihatsu' 'maxus' 'jeep' 'ford' 'shineray' 'morgan' 'scania'
 'aston martin' 'polestar' 'ud' 'sunbeam' 'higer' 'sokon' 'austin' 'lotus'
 'alpine' 'foton' 'ds' 'alfa romeo' 'proton' 'ora' 'international'
 'golden dragon' 'daimler' 'king long' 'triumph' 'yutong' 'dodge' 'srm'
 'man' 'cupra' 'saab' 'camc' 'smart' 'hummer' 'rover' 'datsun' 'chrysler'
 'mitsuoka' 'ruf' 'kyc' 'sc' 'range' 'cadillac' 'bertone' 'valiant' 'aion'
 'joylong' 'sinotruk' 'pontiac' 'td']
Unique Models: ['range' 'c200' 'odyssey' 'altis' 'gs' 's2000' 'huracan' 'avante'
 'evolution' 'c180' 'i

In [14]:
# Check manufactured year
# convert year to type int
df_train['manufactured'] = pd.to_numeric(df_train['manufactured'], errors='coerce').astype('Int64')
print(df_train['manufactured'].describe())

# Check for future years
current_year = pd.Timestamp.now().year
if (df_train['manufactured'] > current_year).any():
    print("There are future years of manufacture")
else:
    print("There are no future years of manufacture")

count        24993.0
mean     2015.645181
std         5.723515
min           1939.0
25%           2014.0
50%           2017.0
75%           2019.0
max           2024.0
Name: manufactured, dtype: Float64
There are no future years of manufacture


In [15]:
# Convert date columns to datetime format (assuming they are in string format)
df_train['original_reg_date'] = pd.to_datetime(df_train['original_reg_date'], errors='coerce')
df_train['reg_date'] = pd.to_datetime(df_train['reg_date'], errors='coerce')

# Check for Missing Values
missing_original_reg_date = df_train['original_reg_date'].isnull().sum()
missing_reg_date = df_train['reg_date'].isnull().sum()

# Check Data Types
original_reg_date_type = df_train['original_reg_date'].dtype
reg_date_type = df_train['reg_date'].dtype

# Check Validity of Dates
future_dates_original_reg = df_train[df_train['original_reg_date'] > pd.Timestamp.now()]
future_dates_reg = df_train[df_train['reg_date'] > pd.Timestamp.now()]

# Check Logical Consistency
inconsistent_dates = df_train[df_train['reg_date'] < df_train['original_reg_date']]

# Output results
print(f"Missing values in 'original_reg_date': {missing_original_reg_date}")
print(f"Missing values in 'reg_date': {missing_reg_date}")
print(f"Data type of 'original_reg_date': {original_reg_date_type}")
print(f"Data type of 'reg_date': {reg_date_type}")

if not future_dates_original_reg.empty:
    print(f"Future dates found in 'original_reg_date': {future_dates_original_reg.shape[0]} entries")
else:
    print("No future dates found in 'original_reg_date'.")

if not future_dates_reg.empty:
    print(f"Future dates found in 'reg_date': {future_dates_reg.shape[0]} entries")
else:
    print("No future dates found in 'reg_date'.")

if not inconsistent_dates.empty:
    print(f"Inconsistent dates found: {inconsistent_dates.shape[0]} entries where 'reg_date' is earlier than 'original_reg_date'.")
else:
    print("All dates are consistent: 'reg_date' is not earlier than 'original_reg_date'.")


/var/folders/j6/tvh87n0s6l12hjnnwjrp9mv00000gn/T/ipykernel_91859/3081706499.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['original_reg_date'] = pd.to_datetime(df_train['original_reg_date'], errors='coerce')
/var/folders/j6/tvh87n0s6l12hjnnwjrp9mv00000gn/T/ipykernel_91859/3081706499.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train['reg_date'] = pd.to_datetime(df_train['reg_date'], errors='coerce')


Missing values in 'original_reg_date': 24745
Missing values in 'reg_date': 0
Data type of 'original_reg_date': datetime64[ns]
Data type of 'reg_date': datetime64[ns]
No future dates found in 'original_reg_date'.
No future dates found in 'reg_date'.
All dates are consistent: 'reg_date' is not earlier than 'original_reg_date'.


In [16]:
# Check for Missing Values
missing_vehicle_type = df_train['type_of_vehicle'].isnull().sum()
missing_transmission = df_train['transmission'].isnull().sum()

# Check Unique Values
unique_vehicle_types = df_train['type_of_vehicle'].unique()
unique_transmissions = df_train['transmission'].unique()

# Check for Consistency
# Define expected categories
expected_vehicle_types = ['suv', 'luxury sedan', 'mpv', 'mid-sized sedan', 'sports car', 'truck', 'hatchback', 'stationwagon', 'bus/mini bus', 'van']

# Check if unique values are in expected categories
inconsistent_vehicle_types = [v for v in unique_vehicle_types if v not in expected_vehicle_types]

# Output results
print(f"Missing values in 'type_of_vehicle': {missing_vehicle_type}")
print(f"Missing values in 'transmission': {missing_transmission}")

print(f"Unique values in 'type_of_vehicle': {unique_vehicle_types}")
print(f"Unique values in 'transmission': {unique_transmissions}")

if inconsistent_vehicle_types:
    print(f"Inconsistent vehicle types found: {inconsistent_vehicle_types}")
else:
    print("All vehicle types are consistent with expected categories.")

# Check that vehicle type is consistent with all model-make pairs
unique_vehicle_count = (df_train
                        .groupby(['make', 'model'])['type_of_vehicle']
                        .nunique()
                        .reset_index(name='unique_count'))

# Step 2: Filter to find combinations with more than one unique vehicle type
multiple_types = unique_vehicle_count[unique_vehicle_count['unique_count'] > 1]

# Step 3: Check if there are any such combinations
if not multiple_types.empty:
    print("The following model and make combinations map to more than one vehicle type:")
    print(multiple_types)
else:
    print("All model and make combinations map to a single vehicle type.")

Missing values in 'type_of_vehicle': 0
Missing values in 'transmission': 0
Unique values in 'type_of_vehicle': ['suv' 'luxury sedan' 'mpv' 'mid-sized sedan' 'sports car' 'truck'
 'hatchback' 'stationwagon' 'bus/mini bus' 'van' 'others']
Unique values in 'transmission': ['auto' 'manual']
Inconsistent vehicle types found: ['others']
The following model and make combinations map to more than one vehicle type:
           make   model  unique_count
21         Audi      A3             3
22         Audi      A4             3
23         Audi      A5             2
24         Audi      A6             2
27         Audi  E-Tron             3
..          ...     ...           ...
770  Volkswagen  Beetle             3
771  Volkswagen   Caddy             2
775  Volkswagen    Golf             4
782  Volkswagen  Passat             3
804       Volvo    Xc40             2

[137 rows x 3 columns]


###    Textual Features Extraction

In [17]:
categorical_columns = df_train.select_dtypes(include=["object"]).columns
categorical_columns

Index(['title', 'make', 'model', 'description', 'type_of_vehicle', 'category',
       'transmission', 'fuel_type', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories'],
      dtype='object')

In [18]:
text_features = ['title', 'description', 'category', 'features', 'accessories', 'opc_scheme', 'eco_category']

In [ ]:
# Create a word cloud for each text feature
for feature in text_features:
    text_data = ' '.join(df_train[feature].dropna().tolist())
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text_data)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  
    plt.title(f'Wordcloud – {feature}')
    plt.savefig(f'visualisations/wordcloud_{feature}.png')
    plt.show()

### Data Cleaning & Data Transformation

In [20]:
## Create binary variables from text features

# Convert opc scheme to binary for further analysis
df_train['opc_scheme'] = df_train['opc_scheme'].apply(lambda x: 1 if pd.notna(x) else 0)
# Create column for parf v coe cars 
df_train['parf'] = df_train['category'].apply(lambda x: 1 if 'parf' in x else 0)
# Create column for rare & exotic cars
df_train['rare'] = df_train['category'].apply(lambda x: 1 if 'rare & exotic' in x else 0)
# Create column for vintage cars
df_train['vintage'] = df_train['category'].apply(lambda x: 1 if 'vintage' in x else 0)

In [21]:
binary_columns = ['opc_scheme', 'parf', 'rare', 'vintage']
for col in binary_columns:
    print(f"\n{col}:")
    print(df_train[col].value_counts().head())


opc_scheme:
opc_scheme
0    24838
1      162
Name: count, dtype: int64

parf:
parf
1    17007
0     7993
Name: count, dtype: int64

rare:
rare
0    24070
1      930
Name: count, dtype: int64

vintage:
vintage
0    24839
1      161
Name: count, dtype: int64


In [22]:
print("\n=== Fix Missing Values (make) ===")
print("Missing values: ",df_train['make'].isna().sum())


df_train['make']          = df_train['make'].str.upper()
df_train['make']          = df_train['make'].str.replace(' ','').str.strip()
df_train['title']         = df_train['title'].str.upper()

make_list                   = [make for make in df_train['make'].unique().tolist() if type(make) == str]
make_list.sort()
print("Unique values: ")
print(make_list[:20])
print()

df_train['make_temp']     = df_train['title'].str.split(' ').str[0]
df_train['make_temp_similar']     = df_train.apply(lambda x: difflib.get_close_matches(x['make_temp'], make_list, n=1)[0], axis=1)

df_train['make']          = df_train['make'].fillna(df_train['make_temp'])
df_train                  = df_train.drop(columns = ['make_temp', 'make_temp_similar'])
print("Missing values (after cleaning): ", df_train['make'].isna().sum())


=== Fix Missing Values (make) ===
Missing values:  1316
Unique values: 
['AION', 'ALFAROMEO', 'ALPINE', 'ASTONMARTIN', 'AUDI', 'AUSTIN', 'BENTLEY', 'BERTONE', 'BMW', 'BYD', 'CADILLAC', 'CAMC', 'CHEVROLET', 'CHRYSLER', 'CITROEN', 'CUPRA', 'DAIHATSU', 'DAIMLER', 'DATSUN', 'DFSK']

Missing values (after cleaning):  0


In [23]:
print("\n=== Fix Missing Values (manufactured) ===")
print("Missing values: ",df_train['manufactured'].isna().sum())

df_train['original_reg_date']         = pd.to_datetime(df_train['original_reg_date'], format = "%d-%b-%Y")
df_train['reg_date']                  = pd.to_datetime(df_train['reg_date'], format = "%d-%b-%Y")

df_train['original_reg_date_temp']    = df_train['original_reg_date'].dt.year
df_train['reg_date_temp']             = df_train['reg_date'].dt.year
df_train['manufactured']              = df_train['manufactured'].fillna(df_train[['original_reg_date_temp','reg_date_temp']].min(axis=1))
df_train['manufactured']              = df_train['manufactured'].astype(int).astype(str)
df_train                              = df_train.drop(columns = ['original_reg_date_temp', 'reg_date_temp'])
print("Missing values (after cleaning): " ,df_train['manufactured'].isna().sum())



=== Fix Missing Values (manufactured) ===
Missing values:  7
Missing values (after cleaning):  0


In [24]:
print("\n=== Fix Missing Values (no_of_owners) ===")
print("Missing values: ",df_train['no_of_owners'].isna().sum())
print("\nSummary statistics: ", df_train['no_of_owners'].describe())
df_train['no_of_owners'] = df_train['no_of_owners'].fillna(df_train['no_of_owners'].median())
print("\nMissing values (after cleaning): " ,df_train['no_of_owners'].isna().sum())


=== Fix Missing Values (no_of_owners) ===
Missing values:  18

Summary statistics:  count    24982.000000
mean         2.289328
std          1.489925
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: no_of_owners, dtype: float64

Missing values (after cleaning):  0


In [25]:
# Extract fuel type from category
fuel_keywords = {
    'electric': 'electric',
    'hybrid': 'petrol-electric'
}

def extract_fuel_type(category_text):
    category_text = category_text.lower()  
    for keyword, fuel_type in fuel_keywords.items():
        if keyword in category_text:
            return fuel_type
    return None

# Apply the function to the rows where fuel_type is missing
df_train['fuel_type_category_fill'] = df_train['fuel_type']
print(f"Number of missing values for fuel_type (before): {df_train['fuel_type_category_fill'].isna().sum()}")
df_train.loc[df_train['fuel_type'].isna(), 'fuel_type_category_fill'] = df_train['category'].apply(extract_fuel_type)
print(f"Number of missing values for fuel_type (after): {df_train['fuel_type_category_fill'].isna().sum()}")


Number of missing values for fuel_type (before): 19121
Number of missing values for fuel_type (after): 19121


In [26]:
# Create mapping for fuel type from model make
fuel_type_mapping = df_train.groupby(['make', 'model'])['fuel_type'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
fuel_type_dict = dict(zip(zip(fuel_type_mapping['make'], fuel_type_mapping['model']), fuel_type_mapping['fuel_type']))

# Define a function to fill missing fuel types
def fill_fuel_type(row, fuel_type_dict):
    if pd.isna(row['fuel_type']):
        return fuel_type_dict.get((row['make'], row['model']), None)
    return row['fuel_type']

# Apply the function to fill in missing values
df_train['fuel_type_model_make_fill'] = df_train['fuel_type']
print(f"Number of missing values for fuel_type (before): {df_train['fuel_type_model_make_fill'].isna().sum()}")
df_train.loc[df_train['fuel_type'].isna(), 'fuel_type_model_make_fill']  = df_train.apply(fill_fuel_type, axis=1, fuel_type_dict=fuel_type_dict)
print(f"Number of missing values for fuel_type (after): {df_train['fuel_type_model_make_fill'].isna().sum()}")

df_train['fuel_type'] = df_train['fuel_type_model_make_fill']


Number of missing values for fuel_type (before): 19121
Number of missing values for fuel_type (after): 9618


In [27]:
from datetime import datetime

df_train['AGE-current'] = datetime.now().year - df_train['manufactured'].astype(int)

"""Calculate the remaining years of COE based on the vehicle title and registration dates."""
df_train['title'] = df_train['title'].str.upper()  # Convert titles to uppercase for consistency

# Extract COE expiration year from title if it contains 'COE'
df_train.loc[df_train['title'].str.contains('COE'), 'coe_temp'] = df_train['title'].str.split(' ').str[-1]
df_train['coe_temp1'] = df_train['coe_temp'].str.replace(')', '')  # Clean up extracted year

# Convert the cleaned COE year to datetime
df_train['coe_temp1'] = pd.to_datetime(df_train['coe_temp1'], format="%m/%Y", errors='coerce')

# Calculate the COE end date based on the latest registration date
df_train['coe_temp2'] = df_train[['original_reg_date', 'reg_date']].max(axis=1) + pd.offsets.DateOffset(years=10)
df_train['coe_enddate'] = df_train['coe_temp1'].fillna(df_train['coe_temp2'])  # Use COE date if available

# Calculate remaining age until COE expiration
df_train['AGE-remaining'] = df_train.apply(lambda x: dateutil.relativedelta.relativedelta(x['coe_enddate'], datetime.now()).years, axis=1)

# Clean up temporary columns used for calculations
df_train = df_train.drop(columns=["coe_temp", "coe_temp1", "coe_temp2", "coe_enddate"])


In [28]:
# Extract coe end date from title
df_train['coe_end'] = df_train['title'].str.extract(r'\(COE TILL (\d{2}/\d{4})\)')
df_train['coe_end'] = pd.to_datetime(df_train['coe_end'], format='%m/%Y')

In [29]:
missing_info = get_missing_info(df_train=df_train)
missing_info

,missing_count,missing_perc
original_reg_date,24745,98.98
lifespan,22671,90.68
fuel_type_category_fill,19121,76.48
coe_end,19032,76.13
fuel_type_model_make_fill,9618,38.47
fuel_type,9618,38.47
mileage,5304,21.22
accessories,3813,15.25
power,2640,10.56
road_tax,2632,10.53


## 5 EDA

### Univariate Analysis

#### Numerical Analysis

In [ ]:
# Identify numerical columns
numerical_cols = df_train.select_dtypes(include=['int64', 'float64']).columns.drop(['listing_id'])

print("\n=== Numerical Features Analysis ===")
print(df_train[numerical_cols].describe())

# Check for outliers using IQR method
print("\n=== Outlier Analysis (IQR Method) ===")
for col in numerical_cols:
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df_train[(df_train[col] < (Q1 - 1.5 * IQR)) | (df_train[col] > (Q3 + 1.5 * IQR))][col]
    if len(outliers) > 0:
        print(f"\n{col}:")
        print(f"Number of outliers: {len(outliers)}")
        print(f"Percentage of outliers: {(len(outliers)/len(df_train)*100):.2f}%")

fig, axes = plt.subplots(len(numerical_cols), 2, figsize=(10, 2*len(numerical_cols)))

for i, feature in enumerate(numerical_cols):
    
    # Histogram
    sns.histplot(df_train[feature].dropna(), kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f'Histogram of {feature}')
    axes[i, 0].set_xlabel(feature)
    
    # Box plot
    sns.boxplot(x=df_train[feature].dropna(), ax=axes[i, 1])
    axes[i, 1].set_title(f'Box Plot of {feature}')
    axes[i, 1].set_xlabel(feature)

plt.tight_layout()
plt.savefig("visualisations/distribution_numerical.png", bbox_inches='tight')
plt.show()


In [ ]:
"""Plot correlation matrix of numerical features"""
corr = df_train[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix of Numerical Features')
plt.savefig("visualisations/correlation_matrix.png")
plt.show()

In [32]:
print(corr)

               curb_weight     power  engine_cap  no_of_owners  depreciation  \
curb_weight       1.000000  0.611829    0.726307     -0.013746      0.160003   
power             0.611829  1.000000    0.865975      0.153000      0.674149   
engine_cap        0.726307  0.865975    1.000000      0.145428      0.451648   
no_of_owners     -0.013746  0.153000    0.145428      1.000000      0.013612   
depreciation      0.160003  0.674149    0.451648      0.013612      1.000000   
coe               0.045155  0.298417    0.088011     -0.175762      0.205189   
road_tax          0.603733  0.822907    0.946788      0.311302      0.511066   
dereg_value       0.120011  0.619471    0.327562     -0.181095      0.706500   
mileage          -0.009429 -0.230732   -0.035450      0.433340     -0.251363   
omv               0.250311  0.819098    0.643648      0.123503      0.776646   
arf               0.122122  0.776173    0.508939      0.073937      0.823259   
opc_scheme       -0.034330 -0.052977   -

**Strongest correlations with price**
- dereg_value (0.91)
- arf (0.89)
- omv (0.82)
- depreciation (0.81)
- power (0.70)

Focus on these features for initial modelling effort.


**Moderate correlations with price**
- road_tax (0.52)
- engine_cap (0.44)
- coe (0.35)
- mileage (-0.39)
- rare (0.60)

**Low correlation with price**
- manufactured (0.20)
- curb_weight (0.15)
- no_of_owners (-0.08)

**Strong correlations between features**

- omv and arf (0.94)
- engine_cap and road_tax (0.94)
- power and engine_cap (0.86)

Potential multicollinearity which may impact some models (tree-based models are less sensitive). Choose most relevant features or employ dimensionality reduction methods like PCA.
Feature engineering to create interaction terms between strongly related features (e.g. power * engine cap)






In [33]:
strong_corr_price = ['dereg_value', 'arf', 'omv', 'depreciation', 'power']
moderate_corr_price = ['road_tax', 'engine_cap', 'coe', 'mileage', 'rare', 'AGE-remaining']

### FOLLOWING CELLS ARE COMMENTED OUT DUE TO LONGER RUNTIME

In [34]:
# plt.figure(figsize=(8, 6))
# sns.pairplot(df_train, vars=strong_corr_price + ['price'], hue='parf')
# plt.title('Features with strong corr with Price Pairplot (by parf)')
# plt.savefig("visualisations/pairplot_strong_corr_by_parf.png")
# plt.show()

In [35]:
# plt.figure(figsize=(8, 6))
# sns.pairplot(df_train, vars=strong_corr_price+['price'], hue='rare')
# plt.title('Features with strong corr with Price Pairplot (by rare)')
# plt.savefig("visualisations/pairplot_strong_corr_by_rare.png")
# plt.show()

In [36]:
# plt.figure(figsize=(8, 6))
# sns.pairplot(df_train, vars=moderate_corr_price + ['price'], hue='parf')
# plt.title('Features with strong corr with Price Pairplot (by parf)')
# plt.savefig("visualisations/pairplot_mod_corr_by_parf.png")
# plt.show()

In [37]:
# plt.figure(figsize=(8, 6))
# sns.pairplot(df_train, vars=moderate_corr_price+['price'], hue='rare')
# plt.title('Features with strong corr with Price Pairplot (by rare)')
# plt.savefig("visualisations/pairplot_mod_corr_by_rare.png")
# plt.show()

#### Categorical Analysis

In [38]:
# Identify categorical columns
categorical_cols = df_train.select_dtypes(include=['object']).columns

print("\n=== Categorical Features Analysis ===")
for col in categorical_cols:
    unique_values = df_train[col].nunique()
    print(f"\n{col}:")
    print(f"Number of unique values: {unique_values}")
    if unique_values < 10:  # Only show value counts for columns with few unique values
        print(df_train[col].value_counts().head())



=== Categorical Features Analysis ===

title:
Number of unique values: 7256

make:
Number of unique values: 95

model:
Number of unique values: 799

description:
Number of unique values: 19765

manufactured:
Number of unique values: 71

type_of_vehicle:
Number of unique values: 11

category:
Number of unique values: 245

transmission:
Number of unique values: 2
transmission
auto      23301
manual     1699
Name: count, dtype: int64

fuel_type:
Number of unique values: 5
fuel_type
petrol-electric    10355
diesel              3697
electric             896
petrol               430
diesel-electric        4
Name: count, dtype: int64

lifespan:
Number of unique values: 1482

eco_category:
Number of unique values: 1
eco_category
uncategorized    25000
Name: count, dtype: int64

features:
Number of unique values: 16769

accessories:
Number of unique values: 16939

fuel_type_category_fill:
Number of unique values: 5
fuel_type_category_fill
diesel             2586
petrol-electric    2265
electri

**Observations**

- eco_category has the same value for all rows. Drop column.
- opc_scheme has 3 unique values. Present value seem to indicate that the car is under OPC scheme. Convert to binary.


#### Categorical Analysis (with added binary features)

In [39]:
print(df_train.columns)
print(categorical_cols)
print(binary_columns)

Index(['listing_id', 'title', 'make', 'model', 'description', 'manufactured',
       'original_reg_date', 'reg_date', 'type_of_vehicle', 'category',
       'transmission', 'curb_weight', 'power', 'fuel_type', 'engine_cap',
       'no_of_owners', 'depreciation', 'coe', 'road_tax', 'dereg_value',
       'mileage', 'omv', 'arf', 'opc_scheme', 'lifespan', 'eco_category',
       'features', 'accessories', 'price', 'parf', 'rare', 'vintage',
       'fuel_type_category_fill', 'fuel_type_model_make_fill', 'AGE-current',
       'AGE-remaining', 'coe_end'],
      dtype='object')
Index(['title', 'make', 'model', 'description', 'manufactured',
       'type_of_vehicle', 'category', 'transmission', 'fuel_type', 'lifespan',
       'eco_category', 'features', 'accessories', 'fuel_type_category_fill',
       'fuel_type_model_make_fill'],
      dtype='object')
['opc_scheme', 'parf', 'rare', 'vintage']


In [40]:
categorical_and_binary_cols = list(categorical_cols) + binary_columns
categorical_and_binary_cols

['title',
 'make',
 'model',
 'description',
 'manufactured',
 'type_of_vehicle',
 'category',
 'transmission',
 'fuel_type',
 'lifespan',
 'eco_category',
 'features',
 'accessories',
 'fuel_type_category_fill',
 'fuel_type_model_make_fill',
 'opc_scheme',
 'parf',
 'rare',
 'vintage']

In [ ]:
MAX_UNIQUE_VALUES = 30
for feature in categorical_and_binary_cols:
    if len(df_train[feature].value_counts()) < MAX_UNIQUE_VALUES:
        plt.figure(figsize=(8, 4))
        sns.countplot(x=feature, data=df_train, order=df_train[feature].value_counts().index, palette="Set3")
        plt.title(f'Distribution of {feature}')
        plt.tight_layout()
        plt.savefig(f'visualisations/distribution_categorical_{feature}.png')
        plt.show()

        
        # Relationship with price
        plt.figure(figsize=(8, 4))
        sns.boxplot(x=feature, y='price', data=df_train, palette="Set3")
        plt.title(f'{feature} vs Price')
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.savefig(f'visualisations/price_vs_{feature}.png')
        plt.show()



**Observations**
- Type of vehicle:
    - SUVs, luxury sedans, and sports cars are the most common vehicle types in the dataset. - SUVs are highly represented, with relatively lower price variability compared to luxury sedans and sports cars.
    - Truck, station wagon, and bus/mini bus have lower counts, indicating that these vehicle types are less represented in this dataset, possibly niche categories. Generally lower median prices.
    - Luxury sedans and sports cars show the highest resale prices, as expected given the premium nature of these vehicles. There are several outliers in these categories, indicating that some cars are priced significantly higher than the rest.
    - SUVs have a relatively wide price range, with a number of outliers in the upper range, possibly high-end or luxury SUVs. 
    
- Tranmission type:
    - The majority of vehicles in the dataset have automatic transmission.
    - Manual transmission vehicles tend to have a lower price range overall compared to automatics. However, there are still some outliers with higher prices, which may correspond to specific models that are rarer or in high demand among enthusiasts (e.g., sports cars with manual transmission).

- Fuel type:
    - The most common fuel type is diesel, followed by petrol-electric and petrol. This suggests a significant preference for diesel vehicles in the dataset.
    - Electric vehicles are less common, while diesel-electric vehicles have the lowest count among the listed fuel types.
    - Petrol-electric vehicles also show a significant range in pricing, with some outliers that could represent premium models. This suggests that hybrid vehicles are valued well in the resale market.
    - Electric vehicles have a narrower price range compared to diesel and petrol-electric, suggesting that the market for used electric vehicles may not be as established yet, potentially affecting their resale value.
    - Note: Many missing values for this feature
- opc_scheme:
    - Since the feature is highly imbalanced, box plot is not very useful
- parf:
    - parf cars have a slightly higher range compared to coe cars. 
- rare:
    - Rare cars can command much higher resale prices. 

In [42]:
date_features = ['original_reg_date','reg_date', 'lifespan', 'coe_end']

In [43]:
print("\n=== Date Fields Analysis ===")
for col in date_features:

    print(f"\n{col}:")
    print(f"Unique values sample: {df_train[col].unique()[:5]}")
    # Check for invalid dates
    try:
        df_train[col] = pd.to_datetime(df_train[col])
        print("Min date:", df_train[col].min())
        print("Max date:", df_train[col].max())
    except:
        print("Error converting to datetime - possible invalid date formats")



=== Date Fields Analysis ===

original_reg_date:
Unique values sample: <DatetimeArray>
[                'NaT', '1981-01-01 00:00:00', '2020-06-23 00:00:00',
 '2014-09-30 00:00:00', '2019-10-31 00:00:00']
Length: 5, dtype: datetime64[ns]
Min date: 1939-05-25 00:00:00
Max date: 2022-11-17 00:00:00

reg_date:
Unique values sample: <DatetimeArray>
['2018-03-08 00:00:00', '2017-12-28 00:00:00', '2007-09-19 00:00:00',
 '2008-12-15 00:00:00', '2006-12-22 00:00:00']
Length: 5, dtype: datetime64[ns]
Min date: 1959-05-06 00:00:00
Max date: 2024-08-30 00:00:00

lifespan:
Unique values sample: [nan '30-may-2039' '11-aug-2031' '09-oct-2032' '13-sep-2037']
Min date: 2023-10-28 00:00:00
Max date: 2044-12-31 00:00:00

coe_end:
Unique values sample: <DatetimeArray>
[                'NaT', '2027-09-01 00:00:00', '2028-12-01 00:00:00',
 '2026-06-01 00:00:00', '2028-11-01 00:00:00']
Length: 5, dtype: datetime64[ns]
Min date: 2024-04-01 00:00:00
Max date: 2034-08-01 00:00:00


/var/folders/j6/tvh87n0s6l12hjnnwjrp9mv00000gn/T/ipykernel_91859/1369164069.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train[col] = pd.to_datetime(df_train[col])


In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(df_train['price'])
plt.title('Boxplot of Price to detect outliers')
plt.show()



In [45]:
df_train[df_train['price']==max(df_train['price'])]

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,transmission,curb_weight,power,fuel_type,engine_cap,no_of_owners,depreciation,coe,road_tax,dereg_value,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,price,parf,rare,vintage,fuel_type_category_fill,fuel_type_model_make_fill,AGE-current,AGE-remaining,coe_end
14485,1290663,ROLLS-ROYCE CULLINAN 6.75A BLACK BADGE,ROLLS-ROYCE,Cullinan,agent unit with warranty. this cullinan black ...,2023,NaT,2023-03-23,suv,"parf car, low mileage car",auto,2710.0,441.0,None,6749.0,1.0,265170.0,109600,8250.0,945555.0,500.0,548890.0,1147560.0,0,NaT,uncategorized,powered by a v12 twin turbo engine producing 5...,"high arf at 1.14m! rolls-royce coachline, besp...",2899999.0,1,0,0,None,None,1,8,NaT
